In [1]:
import numpy as np 
from math import fabs
import matplotlib.pyplot as plt
from warnings import warn
#for part e)
from matrices import heat_flow_function, Pei_function, VFH_function, Poisson_function, wathen_ge, Lehmer_function 
import time 
from scipy.linalg import block_diag


In [2]:
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

In [3]:
def T_tilda(T, gamma, a):
    """
    Implements the correction mentioned in Remark 3. 
    INPUT:
    - T: T_j
    - gamma: gamma_j
    - a: scalar, a or b
    OUTPUT:
    - T_tilda_j: matrix as defined page 76 of Bai et al., for the Gauss-Radau case.
    """
    e_j = np.zeros(T.shape[0])
    e_j[-1] = 1
    delta = np.linalg.solve(T-a*np.eye(T.shape[0]), gamma*gamma*e_j) # tridiagonal system, could be written without np.linalg.solve maybe
    phi = a + delta[-1]
    temp_left = np.vstack((T, gamma*e_j))
    temp_right = np.expand_dims(np.append(gamma*e_j, phi),1)
    return np.hstack((temp_left, temp_right))

def Gerschgorin(A):
    """Simple way to find a and b in R such that the eigenvalues of A are in [a,b].
    INPUT:
    - square matrix A
    OUTPUT:
    - array: [a,b]
    """
    for i in range(A.shape[0]):
        r = 0
        pivot = A[i,i]
        for j in range(A.shape[1]):
            if i!=j:
                r += fabs(A[i][j])

        if i==0:
            a = pivot-r
            b = pivot+r
        else:
            if (pivot-r)<a:
                a = pivot-r
            if (pivot+r)>b:
                b = pivot+r

    return np.array([a, b])    


def check_new_eig(a, eigs, tol=1e-12):
    """
    Checks if a is in array eigs.
    INPUT: 
    - a: number
    - eigs: eigenvalues
    - tol: tolerance 
    OUTPUT:
    - bool: true if a is in eigs, else false.
    """
    for i in range(len(eigs)):
        if np.abs(a-eigs[i])<tol:
            return True
    return False

        
def check_ortho(X, tol=1e-8):
    """
    Checks if the columns of X are orthogonal to one another.
    INPUT:
    - X: 2d matrix, its columns are the ones we want to check the orthogonality of
    OUTPUT:
    - bool: 
        - True: the vectors are orthogonal
        - False: they are not.
    """
    prod = X.T @ X
    for i in range(prod.shape[0]):
        for j in range(prod.shape[1]):
            if i==j:
                if np.abs(prod[i,j]-1)>tol:
                    return False
            else:
                if np.abs(prod[i,j])>tol:
                    return False
    return True


In [58]:
#MODIFICATION ARMELLE

def algorithm_1_AH(A, u, function, maxit=50, epsilon=1e-5):
    # try with another A, check that x_j are orthogonal, check that Gauss works
    '''
    Implements algorithm 1 from Bai et al. It computes a lower/upper bound of the quantity u^T f(A) u by using the Gauss-Radau rules
    INPUT:
    - A: a symmetric positive definite matrix of size n times n for some n, with eigenvalues in [a,b]
    - u: vector of size n 
    - f: smooth function in the interval [a,b]
    - maxit: maximum number of iteration
    - epsilon: tolerance between two iterations
    
    OUTPUT:
    - [U,L]: Upper and Lower bound of the quantity u^T f(A) u by using the Gauss-Radau rule.
    '''
    # Remark 1:
    interval = Gerschgorin(A)
    
    #tried with computing the eigenvalues thx to the np package, not so big a change 
    #eigenvalues=np.linalg.eig(A)[0]
    #interval=[np.min(eigenvalues), np.max(eigenvalues)]
    
    if interval[0]<=0:
        interval[0] = 1e-4
    print("a =", interval[0])
    print("b =", interval[1])
    if (np.linalg.eigvals(A)<=0).any():
        warn("The matrix A should be positive definite.")
        print("eigenvalues of A:", np.linalg.eigvals(A))
    if not (A==A.T).all():
        warn("A is not symmetric. Please choose A such that A=A.T")
        print("A =",A)

    # set the first variables
    x_j2 = np.zeros(u.shape[0])
    x_j1 = u/np.linalg.norm(u)    
    gamma_j = 0.0
    I_j = np.zeros(2)
    I_j1 = np.zeros(2)
    #turning X into a matrix from the start
    n=len(u) 
    X=np.zeros((n, maxit+1))
    X[:,0] = x_j1

    for j in range(maxit):
        print("START j=", j)
        #print("X[j] =", X[j])
        w = A@X[:,j]
        alpha_j=X[:,j].T@w
        r_j = w - alpha_j*X[:,j]
        if j>0:
            r_j = r_j - gamma_j*X[:,j-1]
        
        #reorthogonlization to avoid roundoff error encured by Lanczos
        alphas=X.T@r_j
        r_j=r_j-X@alphas
        alpha_j=alpha_j+alphas[-1]
        gamma_j = np.linalg.norm(r_j)

        # build T_j:
        if j==0:
            T_j = np.array([alpha_j])
        else:
            # print("T_j shape:", T_j.shape)
            # horizontal array [0, ..., 0, gamma_j]
            temp_h = np.expand_dims(np.zeros(T_j.shape[0]),1)
            temp_h[-1] = gamma_j
            # print("temp h shape:",temp_h.shape)
            # vertical array [0, ..., 0, gamma_j, alpha_j].T
            temp_v = np.expand_dims(np.zeros(T_j.shape[0] + 1),1)
            temp_v[-1] = alpha_j
            temp_v[-2] = gamma_j
            # print("temp v shape:", temp_v.shape)
            # new T_j:
            T_j = np.hstack((np.vstack((T_j, temp_h.T)), temp_v))
            #print("T_j:", T_j)
        
        # for Gauss Radau, a or b have to be zeros of the polynomial, i.e. must be eigenvalues of T_tilda_j:
        for i in range(2): # for lower and upper bounds
            T_tilda_j = T_tilda(T_j, gamma_j, interval[i])
            
            #print("T_tilda_j=", T_tilda_j)
            # compute eigenvalues of T_tilda_j: HERE LOOK AT REMARK 6: IT COULD BE SIMPLIFIED!!
            
            #theta_k, eig_vec = np.linalg.eig(T_tilda_j)
            #w_k_square = eig_vec[0, :]*eig_vec[0,:]
            #I_j[i] = np.sum(function(theta_k)*w_k_square)
            
            #simplifying using remark 6
            
            I_j[i] = np.linalg.inv(T_tilda_j)[0,0]
            
        
            #if not check_new_eig(interval[i], np.sort(theta_k)):
            #    warn("The matrix T_tilda_j does not have the right eigenvalues.")
            #    print("interval[", i, "]=", interval[i])
            #    print(np.sort(theta_k))
                
        print("at j=", j,", I_j=", I_j)
        
        if (j>0) & (np.abs(I_j - I_j1) <= epsilon*np.abs(I_j)).all() :
            print("should be all true:")
            print("j>0", j>0)
            print("abs(I_j-I_j1)[0]",np.abs(I_j[0] - I_j1[0]) <= epsilon*np.abs(I_j[0]))
            print("abs(I_j-I_j1)[1]",np.abs(I_j[1] - I_j1[1]) <= epsilon*np.abs(I_j[1]))
            print("Tolerance", epsilon, "was reached at iteration", j+1)
            print("np.abs(I_j-I_j1)=", np.abs(I_j - I_j1))
            print("at j =", j+1, ", I_j=", I_j)
            print("at j =", j+1,", I_j1=", I_j1)
            print("tol*Interval=", epsilon*I_j)
            break
        
        x_j1 = r_j/gamma_j
        X[:,j+1]=x_j1.copy()
        I_j1 = I_j.copy()
        
        print("check that vectors x_j are orthogonal:", check_ortho(X[:,0:j+1])) 
        if not check_ortho(X[:,0:j+1]):
            warn("The algorithm does not build an orthonormal basis, at j ="+str(j))
    return u.dot(u)*I_j   

In [60]:
#the Poisson matrix
k=12
A= Poisson_function(k=k)
n=k**2
print("n=",n)
#u = np.zeros(n)
#u[0] = 1
u = np.random.randn(n)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

n= 144
a = 0.0001
b = 8.0
START j= 0
T_j shape (1,)
T_tilda_j shape (2, 2)
at j= 0 , I_j= [1.87334796e+03 2.90234762e-01]
check that vectors x_j are orthogonal: True
START j= 1
T_j shape (2, 2)
T_tilda_j shape (3, 3)
at j= 1 , I_j= [1.06551337e+03 4.09130558e-01]
check that vectors x_j are orthogonal: True
START j= 2
T_j shape (3, 3)
T_tilda_j shape (4, 4)
at j= 2 , I_j= [261.77060811   0.41475446]
check that vectors x_j are orthogonal: True
START j= 3
T_j shape (4, 4)
T_tilda_j shape (5, 5)
at j= 3 , I_j= [148.03418485   0.43302039]
check that vectors x_j are orthogonal: True
START j= 4
T_j shape (5, 5)
T_tilda_j shape (6, 6)
at j= 4 , I_j= [70.93859089  0.44033066]
check that vectors x_j are orthogonal: True
START j= 5
T_j shape (6, 6)
T_tilda_j shape (7, 7)
at j= 5 , I_j= [43.77660866  0.44628084]
check that vectors x_j are orthogonal: True
START j= 6
T_j shape (7, 7)
T_tilda_j shape (8, 8)
at j= 6 , I_j= [22.95206223  0.44870399]
check that vectors x_j are orthogonal: True
START j=

In [61]:
#the Pei matrix
n=300
alpha=1
A= Pei_function(alpha, n)
#u = np.zeros(n)
#u[0] = 1
u = np.random.randn(n)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

a = 0.0001
b = 301.0
START j= 0
T_j shape (1,)
T_tilda_j shape (2, 2)
at j= 0 , I_j= [9.79989698e+03 9.87427220e-01]
check that vectors x_j are orthogonal: True
START j= 1
T_j shape (2, 2)
T_tilda_j shape (3, 3)
at j= 1 , I_j= [0.20901233 0.20901233]
check that vectors x_j are orthogonal: True
START j= 2
T_j shape (3, 3)
T_tilda_j shape (4, 4)
at j= 2 , I_j= [0.20901233 0.20901233]
should be all true:
j>0 True
abs(I_j-I_j1)[0] True
abs(I_j-I_j1)[1] True
Tolerance 0.01 was reached at iteration 3
np.abs(I_j-I_j1)= [0. 0.]
at j = 3 , I_j= [0.20901233 0.20901233]
at j = 3 , I_j1= [0.20901233 0.20901233]
tol*Interval= [0.00209012 0.00209012]
bounds [59.73692689 59.73692689]
exact value: 282.2123865803073
L<I_ex: True
U>I_ex: False


In [68]:
#the heat flow matrix
n=10
nu=0.2
A= heat_flow_function(nu,n)
#u = np.zeros(n)
#u[0] = 1
N=n**2
u = np.random.randn(N)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

a = 1.0
b = 2.6
START j= 0
T_j shape (1,)
T_tilda_j shape (2, 2)
at j= 0 , I_j= [0.56213354 0.53989857]
check that vectors x_j are orthogonal: True
START j= 1
T_j shape (2, 2)
T_tilda_j shape (3, 3)
at j= 1 , I_j= [0.54477875 0.54385521]
check that vectors x_j are orthogonal: True
START j= 2
T_j shape (3, 3)
T_tilda_j shape (4, 4)
at j= 2 , I_j= [0.54398927 0.54395468]
should be all true:
j>0 True
abs(I_j-I_j1)[0] True
abs(I_j-I_j1)[1] True
Tolerance 0.01 was reached at iteration 3
np.abs(I_j-I_j1)= [7.89473277e-04 9.94629233e-05]
at j = 3 , I_j= [0.54398927 0.54395468]
at j = 3 , I_j1= [0.54477875 0.54385521]
tol*Interval= [0.00543989 0.00543955]
bounds [51.35000249 51.3467368 ]
exact value: 51.71159785727545
L<I_ex: True
U>I_ex: False


In [70]:
#the VFH matrix (issue!!! To be fixed, not positive definite :( 
K=3
A= VFH_function(K)
#u = np.zeros(n)
#u[0] = 1
N=5**K
u = np.random.randn(N)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

a = 0.0001
b = 1.0
eigenvalues of A: [-5.50348401e+00+0.00000000e+00j -5.40614268e+00+0.00000000e+00j
  1.17335074e-01+0.00000000e+00j -5.12479276e+00+0.00000000e+00j
 -5.13917843e+00+0.00000000e+00j -3.19298295e+00+0.00000000e+00j
 -2.47781601e-01+0.00000000e+00j -2.48946435e+00+0.00000000e+00j
 -6.18642810e-01+0.00000000e+00j -5.25502485e+00+0.00000000e+00j
 -5.44939794e+00+0.00000000e+00j -5.44939794e+00+0.00000000e+00j
 -5.44939794e+00+0.00000000e+00j -5.34227452e+00+0.00000000e+00j
 -5.34227452e+00+0.00000000e+00j -5.13237295e+00+0.00000000e+00j
 -5.13237295e+00+0.00000000e+00j -5.34227452e+00+0.00000000e+00j
 -5.13237295e+00+0.00000000e+00j -5.20367825e+00+0.00000000e+00j
 -5.20367825e+00+0.00000000e+00j -5.20367825e+00+0.00000000e+00j
 -3.57422550e-04+0.00000000e+00j -3.57422550e-04+0.00000000e+00j
 -3.57422550e-04+0.00000000e+00j -3.34598777e+00+0.00000000e+00j
 -3.34598777e+00+0.00000000e+00j -3.34598777e+00+0.00000000e+00j
 -1.24435144e+00+0.00000000e+00j -9.43389183e-01+0.00

C:\Users\HOURS\AppData\Local\Temp\ipykernel_31544\167516866.py:29: UserWarning: The matrix A should be positive definite.
  warn("The matrix A should be positive definite.")


In [79]:
#the VFH matrix (issue!!! To be fixed, not positive definite :( 
nx=2
ny=3
A= wathen_ge(nx, ny)
#u = np.zeros(n)
#u[0] = 1
N=3*nx*ny+2*nx+2*ny+1
u = np.random.randn(N)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

a = 0.0001
b = 8126.78958693015
START j= 0
T_j shape (1,)
T_tilda_j shape (2, 2)
at j= 0 , I_j= [7.07967294e+03 1.56205371e-03]
check that vectors x_j are orthogonal: True
START j= 1
T_j shape (2, 2)
T_tilda_j shape (3, 3)
at j= 1 , I_j= [7.03332456e+02 1.47903911e-03]
check that vectors x_j are orthogonal: True
START j= 2
T_j shape (3, 3)
T_tilda_j shape (4, 4)
at j= 2 , I_j= [5.78676888e+02 1.58884992e-03]
check that vectors x_j are orthogonal: True
START j= 3
T_j shape (4, 4)
T_tilda_j shape (5, 5)
at j= 3 , I_j= [2.35433137e+02 1.62660248e-03]
check that vectors x_j are orthogonal: True
START j= 4
T_j shape (5, 5)
T_tilda_j shape (6, 6)
at j= 4 , I_j= [7.91292543e+01 1.64018166e-03]
check that vectors x_j are orthogonal: True
START j= 5
T_j shape (6, 6)
T_tilda_j shape (7, 7)
at j= 5 , I_j= [7.19964145e+00 1.64139700e-03]
check that vectors x_j are orthogonal: True
START j= 6
T_j shape (7, 7)
T_tilda_j shape (8, 8)
at j= 6 , I_j= [3.02946126e+00 1.64227046e-03]
check that vectors x

In [80]:
k=10
A=Lehmer_function(k)
N=k
u = np.random.randn(N)
def f(x):
    return 1/x
tol = 1e-2
L = algorithm_1_AH(A=A, u=u, function=f, maxit=50, epsilon=tol)

print("bounds", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )

a = 0.0001
b = 6.373809523809524
START j= 0
T_j shape (1,)
T_tilda_j shape (2, 2)
at j= 0 , I_j= [5.80918887e+03 1.03610041e+00]
check that vectors x_j are orthogonal: True
START j= 1
T_j shape (2, 2)
T_tilda_j shape (3, 3)
at j= 1 , I_j= [180.22177119   0.63142301]
check that vectors x_j are orthogonal: True
START j= 2
T_j shape (3, 3)
T_tilda_j shape (4, 4)
at j= 2 , I_j= [4.69353541 0.63099568]
check that vectors x_j are orthogonal: True
START j= 3
T_j shape (4, 4)
T_tilda_j shape (5, 5)
at j= 3 , I_j= [0.65303067 0.63099196]
check that vectors x_j are orthogonal: True
START j= 4
T_j shape (5, 5)
T_tilda_j shape (6, 6)
at j= 4 , I_j= [0.69616035 0.63105088]
check that vectors x_j are orthogonal: True
START j= 5
T_j shape (6, 6)
T_tilda_j shape (7, 7)
at j= 5 , I_j= [0.6324409  0.63105324]
check that vectors x_j are orthogonal: True
START j= 6
T_j shape (7, 7)
T_tilda_j shape (8, 8)
at j= 6 , I_j= [0.63112079 0.63105349]
should be all true:
j>0 True
abs(I_j-I_j1)[0] True
abs(I_j-I_j1